In [0]:
library(R6)

Agent <- R6Class("Agent",
                 
                 public = list(
                   bid = NULL,
                   book = NULL,
                   greeting = "Hi! This is Kurt & Mel. How are you doing?",
                   id = NULL,
                   opponent_id = NULL,
                   round = NULL, 
                   response = NULL,
                   
                   set_book = function(book=NA) {
                     self$book <- book
                   },
                   
                   set_id = function(id=NA) {
                     self$id = id
                   },
                   
                   set_opponent_id = function(opponent_id=NA) {
                     self$opponent_id = opponent_id
                   },
                   
                   set_response = function(response=NA) {
                     self$response <-response 
                   },
                   
                   set_round = function(round=NA) {
                     self$round <- round
                   },
                   
                   #Definition function that determines our bid
                   get_bid = function(){
                     
                     #set easy to work with local variable names
                     own_id <- self$id
                     opponent_id <- self$opponent_id
                     book <- self$book
                     
                     #if response is "Lemon!." then always defect
                     if (self$response == "Lemon!."){
                       bid = "defect"
                       
                       #check whether the opponent exists in the book (has already participated in the tournament up until now)
                       #if this is NOT the case, then always cooperate...
                     } else if ((opponent_id %in% book$id1 | opponent_id %in% book$id2) == FALSE){
                       bid = "cooperate"
                       #...otherwise check whether we have previously interacted with the opponent
                     } else if (nrow(subset(book,(id1 == own_id | id1 == opponent_id) & (id2 ==own_id | id2 == opponent_id))) != 0){                       

                       #Definition Tit-for-Tat function
                       titfortat = function(book,own_id,opponent_id){

                         #determine the opponent's bids all the previous times we interacted
                         result1 = book[(book$id1==opponent_id & book$id2==own_id),c("round","bid1")]
                         result2 = book[(book$id1==own_id & book$id2==opponent_id),c("round","bid2")]
                         
                         #determine our own bids all the previous times we interacted
                         result3 = book[(book$id1==own_id & book$id2==opponent_id),c("round","bid1")]
                         result4 = book[(book$id1==opponent_id & book$id2==own_id),c("round","bid2")]
                         
                         #change the names of bid1 and bid2 into bid
                         names(result1) = c("round","bid")
                         names(result2) = c("round","bid")
                         names(result3) = c("round","bid")
                         names(result4) = c("round","bid")
                         
                         #create two tables: one with all of our opponent's bids, and one with all of our own
                         all_opponent_bids = rbind(result1,result2)
                         all_own_bids = rbind(result3,result4)
                         
                         #look for the last bids in each of the tables
                         round_opponent_bids = all_opponent_bids$round 
                         round_own_bids = all_own_bids$round
                         last_round_opponent_bids = max(round_opponent_bids)
                         last_round_own_bids = max(round_own_bids)
                         
                         #save the last bids in variables
                         opponent_last_bid = all_opponent_bids[ all_opponent_bids$round==last_round_opponent_bids,"bid"]
                         own_last_bid = all_own_bids[all_own_bids$round==last_round_own_bids,"bid"]
                         
                         #execute strategy
                         if(opponent_last_bid=="defect"){
                           #insert randomness: in 5% of all cases we incur a defect, we are forgiving (i.e. we return a cooperate instead of defecting)
                           if (runif(1)<0.05){ 
                             bid="cooperate"
                           } else {
                             bid="defect"
                           }
                         } else {
                           bid="cooperate"
                         }
                         return(bid)
                       }
                       #call function executing tit for stat strategy which returns bid
                       bid = titfortat(book,own_id,opponent_id)

                       #if all of the above is FALSE, then our opponent must have already interacted with other agents but not yet with us.
                     } else {

                       #Definition function that analyzes opponent's behavior and formulates bid accordingly
                       opponent_behavior = function(book,opponent_id){
                         #determine the opponent's history of actions
                         opponent_history <- subset(book, (id1 == opponent_id | id2 == opponent_id))
                         
                         #create variables that can be used to determine the number of times opponent has cooperated or defected in previous rounds
                         coop_bid1 = nrow(subset(opponent_history,(id1 == opponent_id) & (bid1=="cooperate")))
                         coop_bid2 = nrow(subset(opponent_history,(id2 == opponent_id) & (bid2=="cooperate")))
                         defect_bid1 = nrow(subset(opponent_history,(id1 == opponent_id) & (bid1=="defect")))
                         defect_bid2 = nrow(subset(opponent_history,(id2 == opponent_id) & (bid2=="defect")))
                         
                         #calculate the total number of times opponent has decided to cooperate vs defect
                         total_coop = coop_bid1 + coop_bid2
                         total_defect = defect_bid1 + defect_bid2
                         total_rounds = total_coop + total_defect
                         
                         #calculate fractions (percentages) of cooperate vs defect
                         fraction_coop = total_coop / total_rounds
                         fraction_defect = total_defect / total_rounds
                         
                         #calculate coop:defect ratio (a score of higher than 1 indicates an agent that tends to be cooperative)
                         coop_defect_ratio = total_coop / total_defect
                         
                         #determine the number of times an opponent has previously cheated its co-player (i.e. opponent defects and co-player cooperates)
                         cheat1 = nrow(subset(opponent_history,((id1 == opponent_id) & (bid1=="defect") & (bid2=="cooperate"))))
                         cheat2 = nrow(subset(opponent_history,((id2 == opponent_id) & (bid2=="defect") & (bid1=="cooperate"))))
                         total_cheat = cheat1 + cheat2
                         
                         #calculate fraction of cheats with regards to total rounds played (the higher the fraction, the more likely the opponent will cheat us)
                         fraction_cheat = total_cheat / total_rounds
                         
                         #combine relevant statistics into a dataframe (for generating a structured output)
                         id <- c(opponent_id)
                         total_rounds <- c(total_rounds)
                         fraction_coop <- c(fraction_coop)
                         fraction_defect <- c(fraction_defect)
                         coop_defect_ratio <- c(coop_defect_ratio)
                         fraction_cheat <- c(fraction_cheat)
                         output = data.frame(id, total_rounds, fraction_coop, fraction_defect, fraction_cheat) 
                         
                         #use output to determine the final bid. Open to multiple options. Analyze if time
                         if (output$fraction_coop > 0.5){
                           bid = 'cooperate'
                         } else {
                           bid = 'defect'
                         }
                         return(bid)
                       }
                       #call function executing the above function
                       bid = opponent_behavior(book,opponent_id)
                     }
                     self$bid <- bid
                   }
                 )
)